In [1]:
from GRASP import GRASP

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from shapely.geometry import Polygon
from shapely.ops import unary_union

def compute_overlap_area(polygons):
    """
    Compute the overlap area between multiple polygons.
    """
    if len(polygons) < 4:
        return 0

    # Create Shapely Polygon objects
    shapely_polygons = []
    for polygon in polygons:
        poly = Polygon(polygon)
        if not poly.is_valid:
            poly = poly.buffer(0)  # Attempt to fix the invalid polygon
        shapely_polygons.append(poly)

    # Compute the union of all polygons
    union_polygon = unary_union(shapely_polygons)

    # Compute individual areas
    individual_areas = [polygon.area for polygon in shapely_polygons]
    total_individual_area = sum(individual_areas)

    # Compute union area
    union_area = union_polygon.area

    # Compute overlap area
    overlap_area = total_individual_area - union_area

    return overlap_area

In [4]:
def compute_turning_degree(lat_lon_sequence, num_directions=8):
    """
    Compute the turning degree of a sequence of latitude and longitude points,
    and calculate the repeated distance due to turns.
    """
    # Define directions in radians
    directions = np.linspace(0, 2 * np.pi, num_directions, endpoint=False)
    
    # Compute differences between consecutive points
    deltas = np.diff(lat_lon_sequence, axis=0)
    distances = np.linalg.norm(deltas, axis=1)
    angles = np.arctan2(deltas[:, 1], deltas[:, 0])
    
    # Project the angles to the defined directions
    projection_indices = np.digitize(angles, directions) % num_directions
    
    # Count turning points
    turning_points = np.sum(np.abs(np.diff(projection_indices)) > 0)
    
    # Compute total path distance
    total_distance = np.sum(distances)
    
    # Compute repeated distance
    direction_distances = np.zeros(num_directions)
    repeated_distance = 0
    
    for i in range(len(projection_indices)):
        direction = projection_indices[i]
        opposite_direction = (direction + num_directions // 2) % num_directions
        if direction_distances[opposite_direction] > 0:
            repeated_distance += min(distances[i], direction_distances[opposite_direction])
            direction_distances[opposite_direction] -= min(distances[i], direction_distances[opposite_direction])
        direction_distances[direction] += distances[i]
    
    return turning_points, total_distance, repeated_distance

In [5]:
def compute_average_turning_angle(lat_lon_sequence):
    """
    Compute the average turning angle of a path.
    If the input is insufficient or there's any issue, returns 0.
    """
    try:
        if len(lat_lon_sequence) < 3:  # 确保有足够的点来计算两次差分
            return 0  # 不足三个点时返回0

        deltas = np.diff(lat_lon_sequence, axis=0)
        if np.any(np.all(deltas == 0, axis=1)):  # 检查是否有连续重复的点
            return 0  # 存在连续重复点时返回0

        angles = np.arctan2(deltas[:, 1], deltas[:, 0])
        angle_changes = np.diff(angles)
        angle_changes = (angle_changes + np.pi) % (2 * np.pi) - np.pi
        average_turning_angle = np.mean(np.abs(angle_changes))
        return average_turning_angle
    except Exception as e:
        print(f"Error: {e}")  # 打印错误信息
        return 0  # 遇到任何异常时返回0

In [6]:
def compute_path_smoothness(lat_lon_sequence):
    """
    Compute the smoothness of a path using curvature.
    """
    deltas = np.diff(lat_lon_sequence, axis=0)
    angles = np.arctan2(deltas[:, 1], deltas[:, 0])
    angle_changes = np.diff(angles)
    angle_changes = (angle_changes + np.pi) % (2 * np.pi) - np.pi
    curvature = np.sum(np.abs(angle_changes))
    return curvature

In [7]:
def plot_route(lat_lon_sequence, title="Route Plot", plts=None, plt_a=None):
    """
    Plot the route given a sequence of latitude and longitude points.
    """
    lat_lon_sequence = np.array(lat_lon_sequence)
    latitudes = lat_lon_sequence[:, 0]
    longitudes = lat_lon_sequence[:, 1]
    
    if plts is None:
        pltt = plt
        pltt.figure(figsize=(5, 3))
    else:
        fig, axs = plt.subplots(plts[0], plts[1])
        fig.suptitle(title)
        if plts[0] == 1:
            pltt = axs[plt_a[1]]
        else:
            pltt = axs[plt_a[0], plt_a[1]]
    pltt.plot(longitudes, latitudes, marker='o', color='b', label='Route')
    pltt.scatter(longitudes[0], latitudes[0], color='g', s=200, label='Start')
    pltt.scatter(longitudes[-1], latitudes[-1], color='r', s=100, label='End')
    
    for i, (lon, lat) in enumerate(zip(longitudes, latitudes)):
        pltt.text(lon, lat, f'{i}', fontsize=12, ha='right')
    
    pltt.title(title)
    pltt.xlabel('Longitude')
    pltt.ylabel('Latitude')
    pltt.legend()
    pltt.grid(True)
    pltt.show()

In [8]:
from scipy.spatial.distance import directed_hausdorff

def compute_frechet_distance(route1, route2):
    """
    Compute the Fréchet distance between two routes.
    """
    route1 = np.array(route1)
    route2 = np.array(route2)
    
    u = np.vstack((route1, route2))
    frechet_dist = max(directed_hausdorff(route1, route2)[0], directed_hausdorff(route2, route1)[0])
    return frechet_dist

In [9]:
import dataset_config as dataset_config
from tqdm import tqdm
import math
dataConfig = dataset_config.DatasetConfig()

database = dataConfig.database
poi_dict = dataConfig.poi_dict
database_function = dataConfig.database_func
poi_cate_dict = dataConfig.poi_cate_dict

def safe_get(dictionary, key, expected_type, default):
    value = dictionary.get(key, default)
    return value if isinstance(value, expected_type) else default

def run_experiment_cases(case_datasets, exp_replace=True, use_cluster=True):
    # 运行每个实例并收集结果
    results_list = []
    ts_list = []
    database.connect()
    for row in tqdm(case_datasets):
        # if len(row['poi_name_list']) == 0:
        #     continue
        # poi_list = database_function['getIdbyName'](row.get('poi_id_list', []), poi_cate_dict['attraction'])
        # poi_id_list = [i[0] for i in poi_list]
        # poi_id_list = list(set(poi_id_list))
        # not_poi_id_list = []
        # plan_entity = GRASP(
        #     N_c_min=row.get('N_c_min', [1, 2, 2]),
        #     N_c_max=row.get('N_c_max', [10, 3, 2]),
        #     maxIterations=1, 
        #     poi_id_list=poi_id_list, 
        #     route_num=row.get('route_num', 3),
        #     not_poi_list=not_poi_id_list,
        #     use_min_restaurant_gap=row.get('use_min_restaurant_gap', 3) * 3600,
        #     start_day_time=row.get('start_day_time', "09:00:00"),
        #     plan_max_time=row.get('plan_max_time', 12),
        #     tightness_w=row.get('tightness_w', 1.0),
        #     exp_replace=exp_replace,
        #     use_cluster=use_cluster
        # )

        # Default configurations
        default_N_c_min = [1, 2, 2]
        default_N_c_max = [10, 3, 2]
        default_route_num = 3
        default_use_min_restaurant_gap = 3
        default_start_day_time = "09:00:00"
        default_plan_max_time = 12
        default_tightness_w = 1.0

        # Update database retrieval and type checking
        poi_list = database_function['getIdbyName'](row.get('poi_id_list', []), poi_cate_dict['attraction'])
        poi_id_list = [i[0] for i in poi_list]
        poi_id_list = list(set(poi_id_list))

        # Initialize GRASP with safe_get
        plan_entity = GRASP(
            N_c_min=safe_get(row, 'N_c_min', list, default_N_c_min),
            N_c_max=safe_get(row, 'N_c_max', list, default_N_c_max),
            maxIterations=1, 
            poi_id_list=poi_id_list, 
            route_num=safe_get(row, 'route_num', int, default_route_num),
            not_poi_list=[],
            use_min_restaurant_gap=safe_get(row, 'use_min_restaurant_gap', int, default_use_min_restaurant_gap) * 3600 if safe_get(row, 'use_min_restaurant_gap', int, default_use_min_restaurant_gap) < 3600 else safe_get(row, 'use_min_restaurant_gap', int, default_use_min_restaurant_gap),
            start_day_time=safe_get(row, 'start_day_time', str, default_start_day_time),
            plan_max_time=safe_get(row, 'plan_max_time', int, default_plan_max_time),
            tightness_w=safe_get(row, 'tightness_w', float, default_tightness_w),
            exp_replace=exp_replace,
            use_cluster=use_cluster
        )

        results, st, wt, ts, tt, ds = plan_entity.GRASP()
        results_list.append(results)
        ts_list.append(ts)

    # 计算每个实例的指标
    metrics = []
    ts_raw = 0
    for results in results_list:
        results_geos = []
        for result in results:
            results_geos.append([(poi[poi_dict['poi_long']], poi[poi_dict['poi_lat']]) for poi in result])
        # print(results_geos)
        # results_geos = np.asarray(results_geos)

        instance_metrics = []
        overlap_area = max(0, compute_overlap_area(results_geos))
        for i, result in enumerate(results_geos):
            turning_points, total_distance, repeated_distance = compute_turning_degree(result)
            average_turning_angle = compute_average_turning_angle(result)
            smoothness = compute_path_smoothness(result)
            frechet_distance = 0
            if len(results_geos) > 1:
                for j in range(len(results_geos)):
                    if i != j:
                        frechet_distance += compute_frechet_distance(result, results_geos[j])
                frechet_distance /= (len(results_geos) - 1)
            instance_metrics.append({
                "turning_points": turning_points,
                "total_distance": total_distance,
                "repeated_distance": repeated_distance,
                "average_turning_angle": average_turning_angle,
                "smoothness": smoothness,
                "overlap_area": overlap_area,
                "frechet_distance": frechet_distance
            })
        metrics.append(instance_metrics)
        
        for result in results:
            # results_score.append([poi[poi_dict['poi_score']] * math.log2(poi[poi_dict['poi_comment_num']] + 1) for poi in result])
            ts_raw += sum([poi[poi_dict['poi_score']] * math.log2(poi[poi_dict['poi_comment_num']] + 1) for poi in result])

    # 计算平均值
    average_metrics = {
        "turning_points": 0,
        "total_distance": 0,
        "repeated_distance": 0,
        "average_turning_angle": 0,
        "smoothness": 0,
        "overlap_area": 0,
        "frechet_distance": 0,
        "ts": 0,
        "ts_raw": ts_raw
    }

    num_routes = len(metrics) * len(metrics[0])
    for instance_metrics in metrics:
        for route_metrics in instance_metrics:
            average_metrics["turning_points"] += route_metrics["turning_points"]
            average_metrics["total_distance"] += route_metrics["total_distance"]
            average_metrics["repeated_distance"] += route_metrics["repeated_distance"]
            average_metrics["average_turning_angle"] += route_metrics["average_turning_angle"]
            average_metrics["smoothness"] += route_metrics["smoothness"]
            average_metrics["overlap_area"] += route_metrics["overlap_area"]
            average_metrics["frechet_distance"] += route_metrics["frechet_distance"]
    for ts in ts_list:
        for s in ts:
            average_metrics["ts"] += s

    for key in average_metrics:
        average_metrics[key] /= num_routes
        
    database.close()

    return average_metrics

In [10]:
# 取消输出warning
import warnings
warnings.filterwarnings('ignore')

In [11]:
import json
file_name = "../data/6-final_data_glm4air.json"

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

#取前10个case进行测试
cases_dataset = cases_dataset[:10]

avg_metrics = run_experiment_cases(cases_dataset, use_cluster=False)
print("Average metrics:")
print(f"Turning points: {avg_metrics['turning_points']:.2f}")
print(f"Total distance: {avg_metrics['total_distance']:.2f}")
print(f"repeated_distance: {avg_metrics['repeated_distance']:.4f}")
print(f"Average turning angle: {avg_metrics['average_turning_angle']:.2f}")
print(f"Smoothness: {avg_metrics['smoothness']:.2f}")
print(f"Overlap area: {avg_metrics['overlap_area']}")
print(f"Fréchet distance: {avg_metrics['frechet_distance']:.2f}")
print(f"Total score: {avg_metrics['ts']}")
print(f"Total score raw: {avg_metrics['ts_raw']}")
print("\n")

100%|██████████| 10/10 [00:06<00:00,  1.58it/s]

Average metrics:
Turning points: 4.27
Total distance: 0.24
repeated_distance: 0.0355
Average turning angle: 2.02
Smoothness: 11.35
Overlap area: 0.0
Fréchet distance: 0.10
Total score: 133.74880170266266
Total score raw: 232.34818925575595




In [12]:
import json

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

avg_metrics = run_experiment_cases(cases_dataset)
print("Average metrics:")
print(f"Turning points: {avg_metrics['turning_points']:.2f}")
print(f"Total distance: {avg_metrics['total_distance']:.2f}")
print(f"repeated_distance: {avg_metrics['repeated_distance']:.4f}")
print(f"Average turning angle: {avg_metrics['average_turning_angle']:.2f}")
print(f"Smoothness: {avg_metrics['smoothness']:.2f}")
print(f"Overlap area: {avg_metrics['overlap_area']}")
print(f"Fréchet distance: {avg_metrics['frechet_distance']:.2f}")
print(f"Total score: {avg_metrics['ts']}")
print(f"Total score raw: {avg_metrics['ts_raw']}")
print("\n")

  0%|          | 2/2400 [00:00<15:01,  2.66it/s]

100%|██████████| 2400/2400 [18:03<00:00,  2.22it/s] 


Average metrics:
Turning points: 3.30
Total distance: 0.26
repeated_distance: 0.0666
Average turning angle: 1.19
Smoothness: 8.12
Overlap area: 0.00041175455803556515
Fréchet distance: 0.06
Total score: 114.12452879899587
Total score raw: 182.17728623102067




In [13]:
import json

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

avg_metrics = run_experiment_cases(cases_dataset, use_cluster=False)
print("Average metrics:")
print(f"Turning points: {avg_metrics['turning_points']:.2f}")
print(f"Total distance: {avg_metrics['total_distance']:.2f}")
print(f"repeated_distance: {avg_metrics['repeated_distance']:.4f}")
print(f"Average turning angle: {avg_metrics['average_turning_angle']:.2f}")
print(f"Smoothness: {avg_metrics['smoothness']:.2f}")
print(f"Overlap area: {avg_metrics['overlap_area']}")
print(f"Fréchet distance: {avg_metrics['frechet_distance']:.2f}")
print(f"Total score: {avg_metrics['ts']}")
print(f"Total score raw: {avg_metrics['ts_raw']}")
print("\n")

100%|██████████| 2400/2400 [15:43<00:00,  2.54it/s] 


Average metrics:
Turning points: 3.05
Total distance: 0.33
repeated_distance: 0.1112
Average turning angle: 1.27
Smoothness: 7.67
Overlap area: 0.0006977949607395149
Fréchet distance: 0.05
Total score: 106.11133900454776
Total score raw: 167.70727813292638




In [14]:
import json

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

avg_metrics = run_experiment_cases(cases_dataset, exp_replace=False)
print("Average metrics:")
print(f"Turning points: {avg_metrics['turning_points']:.2f}")
print(f"Total distance: {avg_metrics['total_distance']:.2f}")
print(f"repeated_distance: {avg_metrics['repeated_distance']:.4f}")
print(f"Average turning angle: {avg_metrics['average_turning_angle']:.2f}")
print(f"Smoothness: {avg_metrics['smoothness']:.2f}")
print(f"Overlap area: {avg_metrics['overlap_area']}")
print(f"Fréchet distance: {avg_metrics['frechet_distance']:.2f}")
print(f"Total score: {avg_metrics['ts']}")
print(f"Total score raw: {avg_metrics['ts_raw']}")
print("\n")

100%|██████████| 2400/2400 [13:09<00:00,  3.04it/s]


Average metrics:
Turning points: 3.13
Total distance: 0.29
repeated_distance: 0.0768
Average turning angle: 1.26
Smoothness: 7.86
Overlap area: 0.0003401172728348556
Fréchet distance: 0.06
Total score: 101.54719333554324
Total score raw: 164.0059956474892




In [15]:
def run_experiment_cases_choose_i(case_datasets, exp_replace=True, use_cluster=True, max_iterations=2):
    # 运行每个实例并收集结果
    results_list = []
    ts_list = []
    database.connect()
    for row in tqdm(case_datasets):
        # if len(row['poi_name_list']) == 0:
        #     continue
        default_N_c_min = [1, 2, 2]
        default_N_c_max = [10, 3, 2]
        default_route_num = 3
        default_use_min_restaurant_gap = 3
        default_start_day_time = "09:00:00"
        default_plan_max_time = 12
        default_tightness_w = 1.0

        # Update database retrieval and type checking
        poi_list = database_function['getIdbyName'](row.get('poi_id_list', []), poi_cate_dict['attraction'])
        poi_id_list = [i[0] for i in poi_list]
        poi_id_list = list(set(poi_id_list))

        # Initialize GRASP with safe_get
        plan_entity = GRASP(
            N_c_min=safe_get(row, 'N_c_min', list, default_N_c_min),
            N_c_max=safe_get(row, 'N_c_max', list, default_N_c_max),
            maxIterations=max_iterations, 
            poi_id_list=poi_id_list, 
            route_num=safe_get(row, 'route_num', int, default_route_num),
            not_poi_list=[],
            use_min_restaurant_gap=safe_get(row, 'use_min_restaurant_gap', int, default_use_min_restaurant_gap) * 3600 if safe_get(row, 'use_min_restaurant_gap', int, default_use_min_restaurant_gap) < 3600 else safe_get(row, 'use_min_restaurant_gap', int, default_use_min_restaurant_gap),
            start_day_time=safe_get(row, 'start_day_time', str, default_start_day_time),
            plan_max_time=safe_get(row, 'plan_max_time', int, default_plan_max_time),
            tightness_w=safe_get(row, 'tightness_w', float, default_tightness_w),
            exp_replace=exp_replace,
            use_cluster=use_cluster
        )
        results, st, wt, ts, tt, ds = plan_entity.GRASP()
        results_list.append(results)
        ts_list.append(ts)

    # 计算每个实例的指标
    metrics = []
    ts_raw = 0
    for results in results_list:
        results_geos = []
        for result in results:
            results_geos.append([(poi[poi_dict['poi_long']], poi[poi_dict['poi_lat']]) for poi in result])
        # print(results_geos)
        # results_geos = np.asarray(results_geos)

        instance_metrics = []
        overlap_area = max(0, compute_overlap_area(results_geos))
        for i, result in enumerate(results_geos):
            turning_points, total_distance, repeated_distance = compute_turning_degree(result)
            average_turning_angle = compute_average_turning_angle(result)
            smoothness = compute_path_smoothness(result)
            frechet_distance = 0
            if len(results_geos) > 1:
                for j in range(len(results_geos)):
                    if i != j:
                        frechet_distance += compute_frechet_distance(result, results_geos[j])
                frechet_distance /= (len(results_geos) - 1)
            instance_metrics.append({
                "turning_points": turning_points,
                "total_distance": total_distance,
                "repeated_distance": repeated_distance,
                "average_turning_angle": average_turning_angle,
                "smoothness": smoothness,
                "overlap_area": overlap_area,
                "frechet_distance": frechet_distance
            })
        metrics.append(instance_metrics)
        
        for result in results:
            # results_score.append([poi[poi_dict['poi_score']] * math.log2(poi[poi_dict['poi_comment_num']] + 1) for poi in result])
            ts_raw += sum([poi[poi_dict['poi_score']] * math.log2(poi[poi_dict['poi_comment_num']] + 1) for poi in result])

    # 计算平均值
    average_metrics = {
        "turning_points": 0,
        "total_distance": 0,
        "repeated_distance": 0,
        "average_turning_angle": 0,
        "smoothness": 0,
        "overlap_area": 0,
        "frechet_distance": 0,
        "ts": 0,
        "ts_raw": ts_raw
    }

    num_routes = len(metrics) * len(metrics[0])
    for instance_metrics in metrics:
        for route_metrics in instance_metrics:
            average_metrics["turning_points"] += route_metrics["turning_points"]
            average_metrics["total_distance"] += route_metrics["total_distance"]
            average_metrics["repeated_distance"] += route_metrics["repeated_distance"]
            average_metrics["average_turning_angle"] += route_metrics["average_turning_angle"]
            average_metrics["smoothness"] += route_metrics["smoothness"]
            average_metrics["overlap_area"] += route_metrics["overlap_area"]
            average_metrics["frechet_distance"] += route_metrics["frechet_distance"]
    for ts in ts_list:
        for s in ts:
            average_metrics["ts"] += s

    for key in average_metrics:
        average_metrics[key] /= num_routes
        
    database.close()

    return average_metrics

In [16]:
import json

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

avg_metrics = run_experiment_cases_choose_i(cases_dataset, exp_replace=False)
print("Average metrics:")
print(f"Turning points: {avg_metrics['turning_points']:.2f}")
print(f"Total distance: {avg_metrics['total_distance']:.2f}")
print(f"repeated_distance: {avg_metrics['repeated_distance']:.4f}")
print(f"Average turning angle: {avg_metrics['average_turning_angle']:.2f}")
print(f"Smoothness: {avg_metrics['smoothness']:.2f}")
print(f"Overlap area: {avg_metrics['overlap_area']}")
print(f"Fréchet distance: {avg_metrics['frechet_distance']:.2f}")
print(f"Total score: {avg_metrics['ts']}")
print(f"Total score raw: {avg_metrics['ts_raw']}")
print("\n")

100%|██████████| 2400/2400 [23:22<00:00,  1.71it/s]  


Average metrics:
Turning points: 3.18
Total distance: 0.29
repeated_distance: 0.0753
Average turning angle: 1.25
Smoothness: 7.93
Overlap area: 0.0003208589388774691
Fréchet distance: 0.06
Total score: 104.20359868011784
Total score raw: 168.22779839501572




In [17]:
import json

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

avg_metrics = run_experiment_cases_choose_i(cases_dataset, exp_replace=False, max_iterations=3)
print("Average metrics:")
print(f"Turning points: {avg_metrics['turning_points']:.2f}")
print(f"Total distance: {avg_metrics['total_distance']:.2f}")
print(f"repeated_distance: {avg_metrics['repeated_distance']:.4f}")
print(f"Average turning angle: {avg_metrics['average_turning_angle']:.2f}")
print(f"Smoothness: {avg_metrics['smoothness']:.2f}")
print(f"Overlap area: {avg_metrics['overlap_area']}")
print(f"Fréchet distance: {avg_metrics['frechet_distance']:.2f}")
print(f"Total score: {avg_metrics['ts']}")
print(f"Total score raw: {avg_metrics['ts_raw']}")
print("\n")

100%|██████████| 2400/2400 [33:35<00:00,  1.19it/s]  


Average metrics:
Turning points: 3.21
Total distance: 0.29
repeated_distance: 0.0762
Average turning angle: 1.26
Smoothness: 8.02
Overlap area: 0.00032146308569445215
Fréchet distance: 0.06
Total score: 105.3658430841116
Total score raw: 169.81371953697905




In [18]:
import json

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

avg_metrics = run_experiment_cases_choose_i(cases_dataset, use_cluster=False, max_iterations=2)
print("Average metrics:")
print(f"Turning points: {avg_metrics['turning_points']:.2f}")
print(f"Total distance: {avg_metrics['total_distance']:.2f}")
print(f"repeated_distance: {avg_metrics['repeated_distance']:.4f}")
print(f"Average turning angle: {avg_metrics['average_turning_angle']:.2f}")
print(f"Smoothness: {avg_metrics['smoothness']:.2f}")
print(f"Overlap area: {avg_metrics['overlap_area']}")
print(f"Fréchet distance: {avg_metrics['frechet_distance']:.2f}")
print(f"Total score: {avg_metrics['ts']}")
print(f"Total score raw: {avg_metrics['ts_raw']}")
print("\n")

100%|██████████| 2400/2400 [28:29<00:00,  1.40it/s] 


Average metrics:
Turning points: 3.08
Total distance: 0.33
repeated_distance: 0.1098
Average turning angle: 1.26
Smoothness: 7.71
Overlap area: 0.0006811245920051493
Fréchet distance: 0.05
Total score: 108.27580124215793
Total score raw: 171.0093945980195


